In [ ]:
#Import libraries

from operator import mod
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical  # convert to one-hot-encoding
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

np.random.seed(2)


In [ ]:
sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# Reading data
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
# preparing data

Y_train = train['label']
X_train = train.drop(labels=['label'], axis=1)

In [ ]:
# Optimizing data

X_train = X_train/255.0
test = test/255.0

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28, 1)  # -1 all values *unlimited*
test = test.values.reshape(-1, 28, 28, 1)

Y_train = to_categorical(Y_train, num_classes=10)

In [ ]:
# Splitting

random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.1, random_state=random_seed)


In [ ]:
# Buildding the model

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))


In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)


In [ ]:
# Complilling model

model.compile(optimizer=optimizer,
              loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# Printing model details

print('Model Details are : ')
print(model.summary())


In [ ]:
epochs = 50
batch_size = 86

In [ ]:
# Fitting model

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_val, Y_val), verbose=2)

In [ ]:
# Printing Accuracy and Loss

ModelLoss, ModelAccuracy = model.evaluate(X_val, Y_val)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy))


In [ ]:
# Predictting test data

y_result = model.predict(test)

print('Prediction Shape is {}'.format(y_result.shape))


In [ ]:
results = np.argmax(y_result,axis = 1)
results = pd.Series(results,name="Label")

In [ ]:
My_submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

My_submission.to_csv("submission.csv",index=False)

In [ ]:
My_submission.head()
